# Optimization Demo

We demonstrate solvers on a knapsack problem with uniform weights.
This means that we have a linear target function and one cardinality constraint (select at most k out of n).

In [1]:
import time
import random
    
n = 20
k = 10

random.seed(25)
utilities = [random.randint(1, 100) for _ in range(n)]
print('Utilities:', utilities)

## Z3

The [`Z3` library](https://github.com/Z3Prover/z3/wiki) offers SMT optimization for various programming languages, besides its SMT solving capabilities.
There are multiple ways to formulate a cardinality constraint.

In [2]:
import z3


variables = z3.Bools(' '.join(['Variable_' + str(i) for i in range(n)]))
optimizer = z3.Optimize()
objective = z3.Sum(*[z3.If(var, val, 0) for var, val in zip(variables, utilities)])
objective = optimizer.maximize(objective)
optimizer.add(z3.AtMost(*variables, k))  # alternative 1
# optimizer.add(z3.PbLe([(x, 1) for x in variables], k))  # alternative 2 (will be transformed to alternative 1)
# optimizer.add(z3.Sum(*[z3.If(x, 1, 0) for x in variables]) <= k)  # alternative 3
start_time = time.perf_counter()
optimizer.check()
end_time = time.perf_counter()
print(f'{n=}, {k=}, time: {end_time-start_time:.2f}s')
print('Objective value:', objective.value())
print('Assignments:', [int(str(optimizer.model()[var]) == 'True') for var in variables])
# print(optimizer)  # internal representation of optimizer

n=20, k=10, time: 0.09s
Objective value: 799
Assignments: [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]


## Google OR-Tools

`Google OR-Tools` also provides a [constraint optimizer](https://developers.google.com/optimization/cp/integer_opt_cp) besides its [solver](https://developers.google.com/optimization/cp/cp_solver) and besides optimizers for other types of problems.
(For example, there is a specialized solver for knapsack problems, bu we want to try a general-purpose solver).
It supports multiple programming languages.
In constrast to `Z3`, its capabilities to formulate constraints are limited.
Though many different constraint types are supported, most constraint types need to be added to the model directly and are stored in different attributes of the model instance, probably to receive a special treatment during optimization.
This prevents arbitrary nesting of expressions.
Only sums, products and weighted sums can be build freely, but for example not logical expressions.

In [3]:
from ortools.sat.python import cp_model


model = cp_model.CpModel()
variables = [model.NewBoolVar('Variable' + str(i)) for i in range(n)]
# objective = sum([var * val for var, val in zip(variables, utilities)])  # builds a nested expression
objective = cp_model.LinearExpr.ScalProd(variables, utilities)  # expression looks more straightforward
model.Maximize(objective)
# constraint = sum(variables) <= k    # builds a nested expression
constraint = cp_model.LinearExpr.Sum(variables) <= k  # same expression, less parantheses
model.Add(constraint)
solver = cp_model.CpSolver()
start_time = time.perf_counter()
status = solver.Solve(model)
end_time = time.perf_counter()
print('Optimal?', status == cp_model.OPTIMAL)
print(f'{n=}, {k=}, time: {end_time-start_time:.2f}s')
print('Objective value:', solver.ObjectiveValue())
print('Assignments:', [solver.Value(var) for var in variables])

Optimal? True
n=20, k=10, time: 0.01s
Objective value: 799.0
Assignments: [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1]


## GEKKO

`GEKKO` mainly provides [optimizers](https://gekko.readthedocs.io/en/latest/) for various kinds of algebraic problems.
With the help of min and max functions, you could also realize simple logical expressions.
To represent expressions, GEKKO might introduce new variables and bind them with constraints.
In fact, expressions are always created from a model instance and are stored there, even if you don't choose to fomally add the constraints.
Also, note that the final assignments might be a little off from their corresponding integers, probably due to numerical issues.

In [4]:
import gekko


model = gekko.GEKKO(remote=False)
variables = [model.Var(lb=0, ub=1, integer=True, name='Variable_' + str(i)) for i in range(n)]
objective = model.sum([var * val for var, val in zip(variables, utilities)])
model.Maximize(objective)
model.Equation(model.sum(variables) <= k)
start_time = time.perf_counter()
model.solve(disp=False)
end_time = time.perf_counter()
print(f'{n=}, {k=}, time: {end_time-start_time:.2f}s')
print('Objective value:', objective.value[0])
print('Assignments:', [var.value[0] for var in variables])
# print(model._objectives)  # internal representation
# for eq in model._equations:
#     print(eq)
# print(model._connections)
# print(model._objects)

n=20, k=10, time: 0.20s
Objective value: 799.00000614
Assignments: [1.6053684221e-11, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0]
